In [1]:
##Importing Libraries
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
from pylab import rcParams
from plotly import tools
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff
from sklearn.metrics import mean_squared_error
from numpy import newaxis
from keras.layers.core import Dense ,Activation,Dropout
from keras.layers.recurrent import LSTM,GRU
from keras.models import Sequential
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.layers import Dense
from tensorflow.keras import losses
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
#User Input Variables
index_reader = 1 #Input for Building
no_of_epochs = 35 #Use only during training

In [6]:
###Making Files for building 
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train['timestamp'] = pd.to_datetime(train['timestamp'],infer_datetime_format=True)  
test['timestamp'] = pd.to_datetime(test['timestamp'],infer_datetime_format=True)  


##Creating training files

for i in range(1,6):
    if(i==1):
        df = train[train['building_number']==1]
        df.index = df['timestamp']
        df = df.resample('1H').mean()
        new_df = df
    else:
        df = train[train['building_number']==i]
        df.index = df['timestamp']
        df = df.resample('1H').mean()
        new_df = pd.concat([new_df,df])
    
#Adding new features
new_df['day of week']=new_df.index.dayofweek 
new_df['Hour']=new_df.index.hour
new_df['corporate'] = new_df['Hour'].apply(lambda x: 0 if 0<= x <= 7 or 20<=x<=23 else 1)
new_df = new_df.drop(['Hour'],axis=1)
 
anomaly_mean_main_meter = new_df['main_meter'].mean()
new_df.loc[(new_df.main_meter > 15000),'main_meter']=anomaly_mean_main_meter
anomaly_mean_sub_meter_1 = new_df['sub_meter_1'].mean()
new_df.loc[(new_df.sub_meter_1 > 5000),'sub_meter_1']=anomaly_mean_sub_meter_1
anomaly_mean_sub_meter_2 = new_df['sub_meter_2'].mean()
new_df.loc[(new_df.sub_meter_2 > 3000),'sub_meter_2']=anomaly_mean_sub_meter_2

one_hot = ['day of week','building_number']
new_df = pd.get_dummies(new_df,columns = one_hot)

#Saving training file
for i in range(1,6):
    new_df[new_df['building_number_'+str(i)]==1].to_csv('./csv_files/intermediate_files/building_'+str(i)+'_train.csv')

##Creating testing files

for i in range(1,6):
    if(i==1):
        df_test = test[test['building_number']==1]
        df_test.index = df_test['timestamp']
        df_test = df_test.resample('1H').mean()
        new_df_test = df_test
    else:
        df_test = test[test['building_number']==i]
        df_test.index = df_test['timestamp']
        df_test = df_test.resample('1H').mean()
        new_df_test = pd.concat([new_df_test,df_test])

#Adding new features
new_df_test['main_meter'] = 'NaN'
new_df_test['sub_meter_1'] = 'NaN'
new_df_test['sub_meter_2'] = 'NaN'
new_df_test['day of week']=new_df_test.index.dayofweek
new_df_test['Hour']=new_df_test.index.hour
new_df_test['corporate'] = new_df_test['Hour'].apply(lambda x: 0 if 0<= x <= 7 or 20<=x<=23 else 1)
new_df_test = new_df_test.drop(['Hour'],axis=1)

one_hot = ['day of week','building_number']
new_df_test = pd.get_dummies(new_df_test,columns = one_hot)

#Saving test file
for i in range(1,6):
    new_df_test[new_df_test['building_number_'+str(i)]==1].to_csv('./csv_files/intermediate_files/building_'+str(i)+'_test.csv')

In [7]:
#Reading Files
dataframe = pd.read_csv('csv_files/intermediate_files/building_'+str(index_reader)+'_train.csv')
shape_old_dataframe = dataframe.shape[0]
test_building = pd.read_csv('csv_files/intermediate_files/building_'+str(index_reader)+'_test.csv')
test_for_index = pd.read_csv('./test.csv')

In [8]:
#Defined Variables
val_time = 1 #1 hour Aggregate
n_input = 72 #Timestep size is 72
n_features = 16 #Input features is 16
val_length = test_building.shape[0] #Prediction file length for particular building
n_pred = 3 
val_length_new = int(float(shape_old_dataframe)*0.2)
val_gap = val_time*4

In [9]:
#Preprocessing Dataframe
dataframe = pd.concat([dataframe,test_building],axis=0)
interval = [dataframe.shape[0]]
print("The shape of raw data is -: ", dataframe.shape)
print("Shape of training data -: ", shape_old_dataframe)
print("Shape of testing data -: ", test_building.shape)
dataframe['timestamp'] = pd.to_datetime(dataframe['timestamp'],infer_datetime_format=True)
dataframe.set_index(dataframe['timestamp'],inplace=True)
dataframe = dataframe.drop(['timestamp'],axis=1)
#Taking particular features
time_series = dataframe[['main_meter', 'sub_meter_1', 'sub_meter_2', 'corporate', 'day of week_0','day of week_1','day of week_2','day of week_3','day of week_4','day of week_5','day of week_6','building_number_1','building_number_2','building_number_3','building_number_4','building_number_5']] 

The shape of raw data is -:  (9182, 17)
Shape of training data -:  6600
Shape of testing data -:  (2582, 17)


In [10]:
#Splitting into training and test data
sum = 0
c = interval[0] 
train_data = time_series[:c-val_length]
test_data = time_series[c-val_length:c]
sum+=c
interval[0] = sum -val_length

test_data = pd.DataFrame(test_data).values.reshape(-1,n_features)
train_data = pd.DataFrame(train_data).values.reshape(-1,n_features)
print("Training data shape ", train_data.shape)
print("Testing data shape ", test_data.shape)

#Normalizing Dataframes
scaler = MinMaxScaler()

scaler.fit(time_series[:interval[0]])
scaled_train_data = scaler.transform(train_data)


#Processing Training data and Testing Data for GRU
X_train = []
y_train = []

for i in range(n_input, interval[0]):
    X_train.append(scaled_train_data[i-n_input:i, :])
    y_train.append(scaled_train_data[i, :n_pred])
    
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], n_input, n_features))

perm = np.random.permutation(X_train.shape[0]) ##Shuffling data
X_train = X_train[perm]
y_train = y_train[perm]

print("Shape of training data after final processing -: ", X_train.shape)

Training data shape  (6600, 16)
Testing data shape  (2582, 16)
Shape of training data after final processing -:  (6528, 72, 16)


In [11]:
#Building GRU Model and Saving Model
val = "building"+str(index_reader)+"_1hr_sigmoid_3meter_adam_1_final"

###Comment out to run the Model
'''
gru_model = Sequential()
gru_model.add(GRU(300,  return_sequences=True, activation='relu',input_shape=(n_input,n_features)))
gru_model.add(GRU(50,activation='relu'))
gru_model.add(Dense(3, activation='sigmoid'))

gru_model.compile(optimizer='adam',loss='mse')

print(gru_model.summary())

#Saving model
model_json = gru_model.to_json()
with open("./final_models/model_GRU_model_all_"+ val +".json", "w") as json_file:
    json_file.write(model_json)

#Fitting Model
gru_model.fit(X_train, y_train, epochs=no_of_epochs, batch_size = 1024)

#Saving weights
gru_model.save_weights("./final_weights/model_GRU_model_all_"+ val +".h5")
print("Saved model to disk")

'''
#Loading Model
json_file = open("./final_models/model_GRU_model_all_"+ val +".json", 'r')
loaded_model_json = json_file.read()
json_file.close()
gru_model = model_from_json(loaded_model_json)
print(gru_model.summary())
# load weights into new model
gru_model.load_weights("./final_weights/model_GRU_model_all_"+ val +".h5")
print("Loaded model from disk")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 72, 300)           286200    
_________________________________________________________________
gru_1 (GRU)                  (None, 50)                52800     
_________________________________________________________________
dense (Dense)                (None, 3)                 153       
Total params: 339,153
Trainable params: 339,153
Non-trainable params: 0
_________________________________________________________________
None
Loaded model from disk


In [12]:
#Predicting from Model
gru_predictions_scaled = list()
batch = scaled_train_data[interval[0]-n_input:interval[0], :]
current_batch = batch.reshape((1,n_input,n_features))
for i in range(0,val_length):   
    gru_pred = ((gru_model.predict(current_batch)).astype(np.float64))
    dummy = test_data[i].copy()
    dummy[:n_pred] = gru_pred
    gru_predictions_scaled.append(dummy)
    current_batch = np.append(current_batch[:,1:],[dummy.reshape(1, -1)], axis=1)
gru_predictions = scaler.inverse_transform(gru_predictions_scaled)
gru_pred = np.array(gru_predictions[:, :n_pred])
gru_pred = np.repeat(gru_pred, val_gap, axis = 0)
final_gru_predictions = pd.DataFrame(gru_pred)
final_gru_predictions.columns = ['main_meter','sub_meter_1','sub_meter_2']
test_for_index = test_for_index[test_for_index['building_number']==index_reader]
final_gru_predictions = final_gru_predictions.iloc[:test_for_index.shape[0],:]
final_gru_predictions.index = test_for_index['timestamp']

In [13]:
#Saving Dataframe into csv
final_gru_predictions.to_csv('csv_files/prediction_files/GRU/building_'+str(index_reader)+'_3meter_gru_predictions.csv')